In [ ]:
from pathlib import Path
import sys
import matplotlib.pyplot as plt

In [ ]:
NB_DIR = Path.cwd()
PROJ_ROOT = NB_DIR.parent
sys.path.append(str(PROJ_ROOT))

In [ ]:
from src.movie_train import train
train_history, val_history = train()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_history, label='Training Loss (MSE)', marker='o')
plt.plot(val_history, label='Validation Loss (MSE)', marker='x')
plt.title('Learning Curve')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.show()